# Agenda: Dates and times

1. Background theory
2. How to turn a column into datetime info
3. Calculations we can perform
4. Datetime columns as our index ("time series")

# Background

When we use the word "time" in our everyday speech, we might mean one of two different things:

1. One specific, unique point in time, specified with year/month/day and hour/minute/second. Of course, you can get even more precise than that. Examples: Time of birth. Time of death. When you graduated university. When a meeting starts. Each of these things is a unique point in time, and you can specify when precisely it happened. When we keep track of these on a computer, they're known as either a `datetime` value or a `timestamp`.
2. A span of time, between two points. This isn't unique, and it doesn't have a specific year/month/day, but we do measure it with the same units. We can use this kind of value to measure a lifespan, or the time spent in school, or the time that people were married, or the time you were in a meeting. The data type used for this is known as a `timedelta` or an `interval`.

Date math:
- `timestamp` - `timestamp` = `timedelta`
- `timestamp` + `timedelta` = `timestamp`
- `timestamp` - `timedelta` = `timestamp`

In [13]:
import pandas as pd
filename = '../data/taxi.csv'

df = pd.read_csv(filename,
                 usecols=['tpep_pickup_datetime',
                          'tpep_dropoff_datetime',
                          'passenger_count',
                          'trip_distance',
                          'total_amount'])
                          

In [14]:
df.head()

,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,total_amount
0,2015-06-02 11:19:29,2015-06-02 11:47:52,1,1.63,17.80
1,2015-06-02 11:19:30,2015-06-02 11:27:56,1,0.46,8.30
2,2015-06-02 11:19:31,2015-06-02 11:30:30,1,0.87,11.00
3,2015-06-02 11:19:31,2015-06-02 11:39:02,1,2.13,17.16
4,2015-06-02 11:19:32,2015-06-02 11:32:49,1,1.40,10.30


In [15]:
df.dtypes

tpep_pickup_datetime      object
tpep_dropoff_datetime     object
passenger_count            int64
trip_distance            float64
total_amount             float64
dtype: object

In [16]:
# the main way to take a string column and get a datetime column based on it
# is with pd.to_datetime, a function that comes with Pandas.

pd.to_datetime(df['tpep_pickup_datetime'])

0      2015-06-02 11:19:29
1      2015-06-02 11:19:30
2      2015-06-02 11:19:31
3      2015-06-02 11:19:31
4      2015-06-02 11:19:32
               ...        
9994   2015-06-01 00:12:59
9995   2015-06-01 00:12:59
9996   2015-06-01 00:13:00
9997   2015-06-01 00:13:02
9998   2015-06-01 00:13:04
Name: tpep_pickup_datetime, Length: 9999, dtype: datetime64[ns]

In [17]:
# how much memory are we *really* using right now on this data frame?
df.memory_usage(deep=True).sum()    # 1,599,972

1599972

In [18]:
# we can assign the datetime version of these columns back to the orignal data frame

df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'])
df['tpep_dropoff_datetime'] = pd.to_datetime(df['tpep_dropoff_datetime'])

In [19]:
df.memory_usage(deep=True).sum()   

400092

Dates and times come in *many* different formats. How did pd.to_datetime know how to convert our strings into datetime values?

- If the datetime value (as a string) looks like `YYYY-MM-DD HH:MM:SS`, then Pandas can handle it just fine.
- If the value is in US format (i.e., `MM-DD-YYYY HH:MM:SS`, then it can also handle it just fine, and by default.
- If the value is in European format (i.e., `DD-MM-YYYY HH:MM:SS`, then you need to pass an argument to `to_datetime`, `dayfirst=True`, which overrides the defaults for US time format.
- If it's in another format, then you have to specify it with a special formatting string that uses lots of `%` characters. These format strings are used in the `strftime` and `strptime` methods for strings in Python (and in many other languages).  You can pass this string as the `date_format` keyword argument to `pd.to_datetime`.

In [21]:
# another (my preferred) way is when we call pd.read_csv
# we can pass an argument, "parse_dates", which takes a list of column names. Those will all be parsed as datetime values.

df = pd.read_csv(filename,
                 usecols=['tpep_pickup_datetime',
                          'tpep_dropoff_datetime',
                          'passenger_count',
                          'trip_distance',
                          'total_amount'],
                parse_dates=['tpep_pickup_datetime',
                             'tpep_dropoff_datetime'])
                          

In [22]:
df.head()

,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,total_amount
0,2015-06-02 11:19:29,2015-06-02 11:47:52,1,1.63,17.80
1,2015-06-02 11:19:30,2015-06-02 11:27:56,1,0.46,8.30
2,2015-06-02 11:19:31,2015-06-02 11:30:30,1,0.87,11.00
3,2015-06-02 11:19:31,2015-06-02 11:39:02,1,2.13,17.16
4,2015-06-02 11:19:32,2015-06-02 11:32:49,1,1.40,10.30


In [23]:
df.dtypes

tpep_pickup_datetime     datetime64[ns]
tpep_dropoff_datetime    datetime64[ns]
passenger_count                   int64
trip_distance                   float64
total_amount                    float64
dtype: object

# Exercise: Chicago weather

1. Read the weather data from Chicago (`../data/chicago,il.csv`)
2. Don't parse the datetime column (the first one). Check to see how much memory it uses as strings.
3. Re-read it using `parse_dates`.
4. Compare the memory usage.

In [25]:
filename = '../data/chicago,il.csv'
df = pd.read_csv(filename)
df.memory_usage(deep=True).sum()  # 332,220

332220

In [27]:
df = pd.read_csv(filename,
                parse_dates=['date_time'])
df.memory_usage(deep=True).sum()  # 288,540

288540

In [28]:
288540 / 332220

0.8685208596713021

In [30]:
filename = '../data/taxi.csv'

df = pd.read_csv(filename,
                 usecols=['tpep_pickup_datetime',
                          'tpep_dropoff_datetime',
                          'passenger_count',
                          'trip_distance',
                          'total_amount'],
                parse_dates=['tpep_pickup_datetime',
                             'tpep_dropoff_datetime'])
                          

In [31]:
df.head()

,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,total_amount
0,2015-06-02 11:19:29,2015-06-02 11:47:52,1,1.63,17.80
1,2015-06-02 11:19:30,2015-06-02 11:27:56,1,0.46,8.30
2,2015-06-02 11:19:31,2015-06-02 11:30:30,1,0.87,11.00
3,2015-06-02 11:19:31,2015-06-02 11:39:02,1,2.13,17.16
4,2015-06-02 11:19:32,2015-06-02 11:32:49,1,1.40,10.30


# How can we extract data from a `datetime` column?

We use the `.dt` accessor, an attribute that is designed to retrieve one part of the `datetime` value. It's very similar in spirit to what we did last week with the `.str` accessor, giving us access to many string functions. Here, we're more typically retrieving pieces of the `datetime`, but there are some methods, too.

In [42]:
df['tpep_pickup_datetime'].dt.is_leap_year

0       False
1       False
2       False
3       False
4       False
        ...  
9994    False
9995    False
9996    False
9997    False
9998    False
Name: tpep_pickup_datetime, Length: 9999, dtype: bool

# How can we use this?

1. Find only those rows in which a particular datetime component is `True`
2. 